In [1]:
import numpy as np
import re
import json
import pandas as pd

In [54]:
model = "text-davinci-003"

labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

merged_df = labels.merge(weights, on='pairid')
merged_df["AtoB"] = ""
merged_df["BtoA"] = ""

with open(model + "_response.json", "r") as file:
    loaded_data = json.load(file)
    
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight,AtoB,BtoA
0,1,Altitude,Temperature,DWD,->,0.166,,
1,2,Altitude,Precipitation,DWD,->,0.166,,
2,3,Longitude,Temperature,DWD,->,0.167,,
3,4,Altitude,Sunshine hours,DWD,->,0.166,,
4,5,Age,Length,Abalone,->,0.143,,


In [22]:
loaded_data[0]['pair0001,_res1']["choices"][0]["text"]

'\n\nYes'

In [58]:
def add_prediction(merged_df, loaded_data, model):
    merged_df = merged_df.set_index('pairid')
    for response in loaded_data:
        pairid, resid, answer = extract_answers(response)
        print(pairid, answer)
#         if answer != 'A/B' and answer[0] != 'E' and (answer[0]=='A' or answer[0]=='B'):
#             answer = answer[0]
#         answer = '->' if answer == 'A' else '<-' if answer == 'B' else answer
        col = "AtoB" if resid == "1" else "BtoA"
        merged_df.loc[int(pairid),col] = answer
#         print("Converted:", answer, "\n")
    return merged_df


def extract_answers(s):
    # Compile regex patterns
    
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),_res(\d)")
    
    # Search for pairid
    pairid_match = pairid_pattern.search(str(s))
    pairid = pairid_match.group(1) if pairid_match else None
    resid = pairid_match.group(2) if pairid_match else None
    
    # Search for answer
    ans = s["pair0"+ str(pairid) + ',_res' + str(resid)]["choices"][0]["text"]
    
    return pairid, resid, ans[2:]

In [67]:
results = add_prediction(merged_df, loaded_data, "gpt-4")

001 Yes
001 No
002 Yes
002 No
003 Yes
003 No
004 Yes
004 No
005 Yes
005 No
006 Yes
006 No
007 Yes
007 No
008 No
008 No
009 Yes
009 No
010 Yes
010 No
011 Yes
011 No
012 No
012 No
013 Yes
013 No
014 Yes
014 No
015 Yes
015 No
016 Yes
016 No
017 No
017 No
018 No
018 No
019 Yes
019 No
020 Yes
020 No
021 Yes
021 No
022 No
022 No
023 No
023 No
024 Yes
024 No
025 Yes
025 No
026 Yes
026 No
027 Yes
027 No
028 Yes
028 No
029 Yes
029 No
030 Yes
030 No
031 Yes
031 No
032 Yes
032 No
033 No
033 No
034 No
034 No
035 Yes
035 No
036 No
036 No
037 No
037 No
038 Yes
038 No
039 No
039 No
040 Yes
040 No
041 Yes
041 No
042 Yes
042 No
043 Yes
043 No
044 Yes
044 No
045 Yes
045 No
046 Yes
046 No
047 No
047 Yes
048 No
048 Yes
049 No
049 Yes
050 No
050 Yes
051 No
051 Yes
052 Yes
052 No
053 Yes
053 Yes
054 Yes
054 Yes
055 Yes
055 Yes
056 No
056 Yes
057 No
057 Yes
058 No
058 Yes
059 No
059 Yes
060 No
060 No
061 No
061 No
062 No
062 No
063 No
063 No
064 No
064 No
065 Yes
065 No
066 No
066 Yes
067 Yes
067 Yes
068 No


In [69]:
results["gt_AtoB"] = np.where(results["groundtruth"] == "->", "Yes", "No")
results["gt_BtoA"] = np.where(results["groundtruth"] == "->", "No", "Yes")
results["correct_AtoB"] = np.where(results["gt_AtoB"] == results["AtoB"], 1, 0)
results["correct_BtoA"] = np.where(results["gt_BtoA"] == results["BtoA"], 1, 0)
wtd_acc = (sum(results["correct_AtoB"]*results["weight"]) + sum(results["correct_BtoA"]*results["weight"])) / (results["weight"].sum()*2)
acc = (sum(results["correct_AtoB"]) + sum(results["correct_BtoA"])) / 108 / 2
print("calculated separately for AtoB and BtoA: ")
print("acc:", acc, "weighted acc:", wtd_acc)

calculated separately for AtoB and BtoA: 
acc: 0.7962962962962963 weighted acc: 0.7844832670477129


In [70]:
results["correct"] = np.where((results["BtoA"] == results["gt_BtoA"]) & (results["gt_AtoB"] == results["AtoB"]), 1, 0)
wtd_acc = (sum(results["correct"]*results["weight"])) / (results["weight"].sum())
acc = (sum(results["correct"])) / 108
print("if we count both correct as correct: ")
print("acc:", acc, "weighted acc:", wtd_acc)

if we count both correct as correct: 
acc: 0.6203703703703703 weighted acc: 0.6031478618079433
